In [315]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
%matplotlib qt

In [317]:
#open files
# workdir='/Users/likkhian/Desktop/waterBudget/chemdsrun/'
# d01=xr.open_dataset(workdir+'d01/wrfout_d01_2008-06-10_00:00:00')
# d02=xr.open_dataset(workdir+'d02/wrfout_d01_2008-06-10_00:00:00')
workdir='/Users/likkhian/OneDrive/Thesis/wrfchem_analysis/dynamicdownscale/'
d01=xr.open_dataset(workdir+'wrfout_d02_2008-08-01_00:00:00')
d02=xr.open_dataset(workdir+'wrfout_d03_2008-08-01_00:00:00')

code to view vars and attrs
cut d03 at south_north,10,495 -d west_east,13,561
cut d02 at south_north,121,282 -d west_east,28,210
cut d01 at south_north,82,135 -d west_east,18,78
print(d01.data_vars)
for ii in d01.data_vars:
    print(ii,d01[ii].attrs)
    print('\n')
print(d01.dims)
HsiangHe secret formula in NSCC: /home/users/academy/smart/hsianghe/NCL/concentration/estimate_tracer_pblh.ncl

In [318]:
#trim to shape
print(d01.dims)
# print(d01.coords)
print(d01.XLAT.shape,d02.XLAT.shape)
print(d02.XLAT_V[0,0,0].data,d01.XLAT_V[0,19,45].data)
print(d02.XLONG_U[0,0,0].data,d01.XLONG_U[0,19,45].data)
print(d02.XLAT_V[0,-1,-1].data,d01.XLAT_V[0,173,173].data)
print(d02.XLONG_U[0,-1,-1].data,d01.XLONG_U[0,173,173].data)

print(d02.P.shape,d01.P[0,:,19:173,45:173].shape)
slat=19
elat=173
slon=45
elon=173

Frozen(SortedKeysDict({'bottom_top': 29, 'Time': 1, 'south_north_stag': 181, 'bottom_top_stag': 30, 'west_east_stag': 185, 'south_north': 180, 'west_east': 184, 'soil_layers_stag': 4}))
(1, 180, 184) (1, 462, 384)
8.148506164550781 8.14849853515625
99.23342895507812 99.23343658447266
20.210052490234375 20.210044860839844
109.59524536132812 109.59524536132812
(1, 29, 462, 384) (29, 154, 128)


In [319]:
def levelHeight(data):
    z0 = (data.PH[0,:,:,:]+data.PHB[0,:,:,:])/9.81
    zb = z0[0:29,:,:]
    zt = z0[1:30,:,:]
    dz = np.subtract(zt,zb)
    return dz
    
def levelPressure(data):
    v,h,y,x=data.P.shape
    dp=np.empty([h,y,x])
    p0=data.P+data.PB
    for i in range(0,28):
        dp[i,:,:]=p0[0,i,:,:]-p0[0,i+1,:,:]
    #subptract the ptop height from the opmost level
    dp[28,:,:]=p0[0,28,:,:]-5000
    return dp
    
def levelDensity(data):
    return levelPressure(data)/levelHeight(data)

In [321]:
dd1=levelDensity(d01)
dd2=levelDensity(d02)
dz1=levelHeight(d01)
dz2=levelHeight(d02)
# print(dd.shape)
# print(dd[:,0,0].data)
d1vapor=dd1[:,slat:elat,slon:elon]*d01.QVAPOR[0,:,slat:elat,slon:elon].data*dz1[:,slat:elat,slon:elon]*9000*9000
d2vapor=dd2*dz2*d02.QVAPOR[0,:,:].data*3000*3000
d1vaporsum=sum(sum(sum(d1vapor))).data+0
d2vaporsum=sum(sum(sum(d2vapor))).data+0
print(type(d1vaporsum))
print('d01,d02 qvapor comparison','{:.4e}'.format(d1vaporsum),'{:.4e}'.format(d2vaporsum),d1vaporsum/d2vaporsum)

d1cloud=dd1[:,slat:elat,slon:elon]*d01.QCLOUD[0,:,slat:elat,slon:elon].data*dz1[:,slat:elat,slon:elon]*9000*9000
d2cloud=dd2*dz2*d02.QCLOUD[0,:,:,:].data*3000*3000
print('d01,d02 qcloud comparison','{:.4e}'.format(sum(sum(sum(d1cloud))).data+0),'{:.4e}'.format(sum(sum(sum(d2cloud))).data+0),sum(sum(sum(d1cloud)).data)/sum(sum(sum(d2cloud))).data)
d1rain=dd1[:,slat:elat,slon:elon]*d01.QRAIN[0,:,slat:elat,slon:elon].data*dz1[:,slat:elat,slon:elon]*9000*9000
d2rain=dd2*dz2*d02.QRAIN[0,:,:,:].data*3000*3000
print('d01,d02 qrain comparison','{:.4e}'.format(sum(sum(sum(d1rain))).data+0),'{:.4e}'.format(sum(sum(sum(d2rain))).data+0),sum(sum(sum(d1rain)).data)/sum(sum(sum(d2rain)).data))
d1ice=dd1[:,slat:elat,slon:elon]*d01.QICE[0,:,slat:elat,slon:elon].data*dz1[:,slat:elat,slon:elon]*9000*9000
d2ice=dd2*dz2*d02.QICE[0,:,:,:].data*3000*3000
print('d01,d02 qice comparison','{:.4e}'.format(sum(sum(sum(d1ice))).data+0),'{:.4e}'.format(sum(sum(sum(d2ice))).data+0),sum(sum(sum(d1ice)).data)/sum(sum(sum(d2ice)).data))
d1snow=dd1[:,slat:elat,slon:elon]*d01.QSNOW[0,:,slat:elat,slon:elon].data*dz1[:,slat:elat,slon:elon]*9000*9000
d2snow=dd2*dz2*d02.QSNOW[0,:,:,:].data*3000*3000
print('d01,d02 qsnow comparison','{:.4e}'.format(sum(sum(sum(d1snow))).data+0),'{:.4e}'.format(sum(sum(sum(d2snow))).data+0),sum(sum(sum(d1snow)).data)/sum(sum(sum(d2snow)).data))
d1graup=dd1[:,slat:elat,slon:elon]*d01.QGRAUP[0,:,slat:elat,slon:elon].data*dz1[:,slat:elat,slon:elon]*9000*9000
d2graup=dd2*dz2*d02.QGRAUP[0,:,:,:].data*3000*3000
print('d01,d02 qraup comparison','{:.4e}'.format(sum(sum(sum(d1graup))).data+0),'{:.4e}'.format(sum(sum(sum(d2graup))).data+0),sum(sum(sum(d1graup)).data)/sum(sum(sum(d2graup)).data))

# print(d01.ac0.shape)
# d1accum=dd1[:,slat:elat,slon:elon]*d01.ac0[0,:,slat:elat,slon:elon].data*dz1[:,slat:elat,slon:elon]*9000*9000
# d2accum=dd2*dz2*d02.ac0[0,:,:,:].data*3000*3000
# print('d01,d02 accumulation mode number','{:.4e}'.format(sum(sum(sum(d1accum))).data+0),'{:.4e}'.format(sum(sum(sum(d2accum))).data+0))
# d1ait=dd1[:,slat:elat,slon:elon]*d01.nu0[0,:,slat:elat,slon:elon].data*dz1[:,slat:elat,slon:elon]*9000*9000
# d2ait=dd2*dz2*d02.nu0[0,:,:,:].data*3000*3000
# print('d01,d02 aitkins mode number','{:.4e}'.format(sum(sum(sum(d1ait))).data+0),'{:.4e}'.format(sum(sum(sum(d2ait))).data+0))

<class 'numpy.float64'>
d01,d02 qvapor comparison 8.4729e+14 8.3694e+14 1.01236529126
d01,d02 qcloud comparison 1.9994e+10 3.3017e+10 0.605559293963
d01,d02 qrain comparison 1.1784e+11 2.5783e+11 0.457066479516
d01,d02 qice comparison 2.8414e+11 1.5335e+11 1.85296296853
d01,d02 qsnow comparison 1.5083e+11 1.2172e+11 1.23919278219
d01,d02 qraup comparison 4.5456e+05 5.6125e+10 8.09907922113e-06


In [322]:
d01ppt=(sum(sum(d01.RAINC[0,slat:elat,slon:elon].data))+sum(sum(d01.RAINNC[0,slat:elat,slon:elon].data)))*9000*9000
d02ppt=(sum(sum(d02.RAINC[0,:,:].data))+sum(sum(d02.RAINNC[0,:,:].data)))*3000*3000
print('d01 rainfall sum ','{:.4e}'.format(d01ppt))
print('d02 rainfall sum ','{:.4e}'.format(d02ppt))
d1overall=sum(sum(sum(d1vapor))).data+sum(sum(sum(d1rain))).data+sum(sum(sum(d1ice))).data+sum(sum(sum(d1snow))).data\
+sum(sum(sum(d1graup))).data
d2overall=sum(sum(sum(d2vapor))).data+sum(sum(sum(d2rain))).data+sum(sum(sum(d2ice))).data+sum(sum(sum(d2snow))).data\
+sum(sum(sum(d2graup))).data
print('d01,d02 ','{:.4e}'.format(d1overall),'{:.4e}'.format(d2overall),d1overall/d2overall)
print('d01,d02 with ppt ','{:.4e}'.format(d1overall+d01ppt),'{:.4e}'.format(d2overall+d02ppt),(d1overall+d01ppt)/(d2overall+d02ppt))

d01 rainfall sum  7.3171e+14
d02 rainfall sum  2.7182e+14
d01,d02  8.4784e+14 8.3753e+14 1.01231337837
d01,d02 with ppt  1.5796e+15 1.1094e+15 1.42385291452


In [ ]:
print(d01.W.shape)
print(d02.W.shape)
plt.figure(1)
plt.pcolor(np.mean(d01.W[0,:,slat:elat,slon:elon],2),vmax=0.1,vmin=-0.1)
plt.colorbar()
plt.figure(2)
plt.pcolor(np.mean(d02.W[0,:,:],2),vmax=0.1,vmin=-0.1)
plt.colorbar()
print('overall average',np.mean(np.mean(np.mean(d01.W[0,:,slat:elat,slon:elon]))),np.mean(np.mean(np.mean(d02.W[0,:,:]))))

In [242]:
print(d02.QCLOUD.shape)

(1, 29, 462, 384)


In [270]:
import math
print(d1graup.data.shape)
print(d01.QNGRAUPEL[0,:,slat:elat,slon:elon].data.shape)
print('total mass of graupel','{:.4e}'.format(sum(sum(sum(d1graup.data)))))
d1graupnum=d1graup.data*d01.QNGRAUPEL[0,:,slat:elat,slon:elon].data
totalnum=sum(sum(sum(d1graupnum)))
print('total number of particles','{:.4e}'.format(totalnum))
graupmass=(sum(sum(sum(d1graup.data)))/totalnum)
print('mass on one particle','{:.4e}'.format(graupmass))
r=(graupmass*3/4/math.pi)**(1/3)/1000
print('radius in m', '{:.4e}'.format(r))
# print(d1graup.shape)
# print(d01.QNGRAUPEL[0,:,slat:elat,slon:elon].data.shape)

print(d2graup.data.shape)
print(d02.QNGRAUPEL.data.shape)
print('total mass of graupel','{:.4e}'.format(sum(sum(sum(d2graup.data)))))
d2graupnum=d2graup.data*d02.QNGRAUPEL[0,:,:,:].data
totalnum=sum(sum(sum(d2graupnum)))+0
print('total number of particles',totalnum)
graupmass=(sum(sum(sum(d2graup.data)))/totalnum)
print('mass on one particle','{:.4e}'.format(graupmass))
r=(graupmass*3/4/math.pi)**(1/3)/1000
print('radius in m', '{:.4e}'.format(r))
# print(d1graup.shape)
# print(d01.QNGRAUPEL[0,:,slat:elat,slon:elon].data.shape)


(29, 154, 128)
(29, 154, 128)
total mass of graupel 1.9045e+10
total number of particles 6.8021e+03
mass on one particle 2.7998e+06
radius in m 8.7434e-02
(29, 462, 384)
(1, 29, 462, 384)
total mass of graupel 7.2105e+11
total number of particles 7.14723114142e+14
mass on one particle 1.0089e-03
radius in m 6.2218e-05


2008-06-11_06:00:00
d01,d02 qvapor comparison 8.8686e+14 9.3679e+14 0.94671018047
d01,d02 qcloud comparison 2.1411e+11 4.1016e+11 0.522008368959
d01,d02 qrain comparison 1.5535e+11 3.3412e+12 0.0464948808609
d01,d02 qice comparison 9.3597e+11 3.4703e+11 2.69706036399
d01,d02 qsnow comparison 7.8703e+11 2.3258e+12 0.338394052298
d01,d02 qraup comparison 6.4217e+05 1.2833e+12 5.0040827849e-07
d01,d02  8.8874e+14 9.4408e+14 0.941382176003

2008-06-11_03:00:00
d01,d02 qvapor comparison 8.8378e+14 9.3258e+14 0.947672602348
d01,d02 qcloud comparison 2.5967e+11 3.2088e+11 0.809237964332
d01,d02 qrain comparison 2.2098e+11 3.2379e+12 0.0682491061435
d01,d02 qice comparison 6.7419e+11 2.6045e+11 2.58853151033
d01,d02 qsnow comparison 7.2362e+11 2.9950e+12 0.241613644615
d01,d02 qraup comparison 1.3669e+07 7.7187e+11 1.77095103057e-05
d01,d02  8.8540e+14 9.3985e+14 0.942069365347

2008-06-11_00:00:00
d01,d02 qvapor comparison 8.8157e+14 9.2803e+14 0.94993056141
d01,d02 qcloud comparison 3.2091e+11 3.5428e+11 0.905790401214
d01,d02 qrain comparison 2.2724e+11 2.9229e+12 0.0777441137405
d01,d02 qice comparison 5.4387e+11 2.0784e+11 2.61675868981
d01,d02 qsnow comparison 6.4746e+11 3.7038e+12 0.174809007882
d01,d02 qraup comparison 1.3307e+07 1.2970e+12 1.02591865303e-05
d01,d02  8.8298e+14 9.3616e+14 0.94319470007

2008-06-10:09:00:00
d01,d02 qvapor comparison 8.8238e+14 9.3010e+14 0.948689973668
d01,d02 qcloud comparison 1.9993e+11 4.3704e+11 0.457465475311
d01,d02 qrain comparison 1.0179e+11 2.0713e+12 0.0491444357155
d01,d02 qice comparison 1.0263e+12 3.2628e+11 3.14558468293
d01,d02 qsnow comparison 6.6192e+11 1.2896e+12 0.513261138584
d01,d02 qraup comparison 1.0399e+05 1.0303e+12 1.00934817741e-07
d01 rainfall sum  3.1482e+14
d02 rainfall sum  1.3134e+14
d01,d02  8.8417e+14 9.3482e+14 0.945817323696
d01,d02 with ppt  1.1990e+15 1.0662e+15 1.1245864171

2008-06-10:06:00:00
d01,d02 qvapor comparison 8.6877e+14 9.1359e+14 0.950945164266
d01,d02 qcloud comparison 1.8773e+11 4.6370e+11 0.404849101966
d01,d02 qrain comparison 4.5871e+10 2.2285e+12 0.0205837751588
d01,d02 qice comparison 8.5724e+11 2.5814e+11 3.32085402874
d01,d02 qsnow comparison 4.6920e+11 1.0228e+12 0.458740500066
d01,d02 qraup comparison 6.0489e+04 5.9476e+11 1.01704017545e-07
d01 rainfall sum  3.1144e+14
d02 rainfall sum  1.2992e+14
d01,d02  8.7014e+14 9.1769e+14 0.948187640293
d01,d02 with ppt  1.1816e+15 1.0476e+15 1.12788178904

2008-06-10_03:00:00
d01,d02 qvapor comparison 8.6421e+14 9.0421e+14 0.955764232812
d01,d02 qcloud comparison 2.5204e+11 3.6826e+11 0.684395826405
d01,d02 qrain comparison 8.1098e+10 2.4058e+12 0.0337094618356
d01,d02 qice comparison 7.2569e+11 1.9511e+11 3.71939408945
d01,d02 qsnow comparison 4.0268e+11 1.8363e+12 0.219292947211
d01,d02 qraup comparison 1.0229e+07 7.4019e+11 1.3819969102e-05
d01 rainfall sum  3.0784e+14
d02 rainfall sum  1.2818e+14
d01,d02  8.6542e+14 9.0939e+14 0.951652860894
d01,d02 with ppt  1.1733e+15 1.0376e+15 1.13077715974

2008-06-10_00:00:00
d01,d02 qvapor comparison 8.6093e+14 9.0084e+14 0.955695911848
d01,d02 qcloud comparison 3.5585e+11 3.9404e+11 0.903091237938
d01,d02 qrain comparison 1.4920e+11 2.4073e+12 0.0619787881706
d01,d02 qice comparison 6.6561e+11 1.6824e+11 3.95627909784
d01,d02 qsnow comparison 5.1702e+11 1.2890e+12 0.401097928864
d01,d02 qraup comparison 1.9045e+10 7.2105e+11 0.0264122587776
d01 rainfall sum  3.0496e+14
d02 rainfall sum  1.2647e+14
d01,d02  8.6228e+14 9.0543e+14 0.952347704762
d01,d02 with ppt  1.1672e+15 1.0319e+15 1.1311614097



In [296]:
1/1.3819969102e-05


72359.06192115016

2008-08-01_00
d01,d02 qvapor comparison 8.66605731478e+14 864497573630086.0
d01,d02 qcloud comparison 183503400026.0 168270554209.40942
d01,d02 qrain comparison 131276905319.0 432547726913.3438
d01,d02 qice comparison 334703711219.0 137636315059.57748
d01,d02 qvapor comparison 176440968914.0 165015554416.94427
d01,d02 qraup comparison 88229.469103 126147625695.06125
8.67248153151e+14 8.65358920852e+14

In [6]:
print('PHB',d01.PHB.attrs)
print('\n')
print('PH',d01.PH.attrs)
print('\n')
print('ALT',d01.ALT.attrs)
print('\n')
print('PBLH',d01.PBLH.attrs)
print('\n')
print('HGT',d01.HGT.attrs)
print('\n')

PHB OrderedDict([('FieldType', 104), ('units', 'm2 s-2'), ('MemoryOrder', 'XYZ'), ('stagger', 'Z'), ('description', 'base-state geopotential')])


PH OrderedDict([('FieldType', 104), ('units', 'm2 s-2'), ('MemoryOrder', 'XYZ'), ('stagger', 'Z'), ('description', 'perturbation geopotential')])


ALT OrderedDict([('FieldType', 104), ('units', 'm3 kg-1'), ('MemoryOrder', 'XYZ'), ('stagger', ''), ('description', 'inverse density')])


PBLH OrderedDict([('FieldType', 104), ('units', 'm'), ('MemoryOrder', 'XY '), ('stagger', ''), ('description', 'PBL HEIGHT')])


HGT OrderedDict([('FieldType', 104), ('units', 'm'), ('MemoryOrder', 'XY '), ('stagger', ''), ('description', 'Terrain Height')])




In [7]:
z0_d01 = (d01.PH[0,:,:,:]+d01.PHB[0,:,:,:])/9.81
z0_d02 = (d02.PH[0,:,:,:]+d02.PHB[0,:,:,:])/9.81
z0_d03 = (d03.PH[0,:,:,:]+d03.PHB[0,:,:,:])/9.81
print('z0_d01 ', z0_d01.shape)

zb_d01 = z0_d01[0:29,:,:]
zb_d02 = z0_d02[0:29,:,:]
zb_d03 = z0_d03[0:29,:,:]
print('zb_d01 ',zb_d01.shape)

zt_d01 = z0_d01[1:30,:,:]
zt_d02 = z0_d02[1:30,:,:]
zt_d03 = z0_d03[1:30,:,:]
print('zt_d01 ',zt_d01.shape)


dz_d01 = np.subtract(zt_d01,zb_d01)
dz_d02 = np.subtract(zt_d02,zb_d02)
dz_d03 = np.subtract(zt_d03,zb_d03)
print('dz_d01 ',dz_d01.shape)

pblhr_d01 = d01.ALT[0,:,:,:] #why do i need this i don't use it!
pblh_d01 = d01.PBLH[0,:,:]
pblhr_d02 = d02.ALT[0,:,:,:] #why do i need this i don't use it!
pblh_d02 = d02.PBLH[0,:,:]
pblhr_d03 = d03.ALT[0,:,:,:] #why do i need this i don't use it!
pblh_d03 = d03.PBLH[0,:,:]

for ii in range(0,29):
    pblhr_d01[ii,:,:] = pblh_d01+d01.HGT[0,:,:]
    pblhr_d02[ii,:,:] = pblh_d02+d02.HGT[0,:,:]
    pblhr_d03[ii,:,:] = pblh_d03+d03.HGT[0,:,:]
print(pblhr_d01.shape)

# for ii in range(0,29):
#     dz_d01[ii,:,:] = (zt_d01[ii,:,:]>pblhr_d01[ii,:,:])*0 + (zt_d01[ii,:,:]>=pblhr_d01[ii,:,:])*dz_d01[ii,:,:]
#     dz_d02[ii,:,:] = (zt_d02[ii,:,:]>pblhr_d02[ii,:,:])*0 + (zt_d02[ii,:,:]>=pblhr_d02[ii,:,:])*dz_d02[ii,:,:]
#     dz_d03[ii,:,:] = (zt_d03[ii,:,:]>pblhr_d03[ii,:,:])*0 + (zt_d03[ii,:,:]>=pblhr_d03[ii,:,:])*dz_d03[ii,:,:]
# print(dz_d01[:,3,10].data)


z0_d01  (30, 54, 61)
zb_d01  (29, 54, 61)
zt_d01  (29, 54, 61)
dz_d01  (29, 54, 61)
(29, 54, 61)


In [9]:
#invert density and multiply by dz and resolution to get air mass
densityinv_d01 = d01.ALT[0,:,:,:]
density_d01 = 1/densityinv_d01
airmass_d01 = np.multiply(density_d01 , dz_d01) * 27000*27000
densityinv_d02 = d02.ALT[0,:,:,:]
density_d02 = 1/densityinv_d02
airmass_d02 = np.multiply(density_d02 , dz_d02) * 9000*9000
densityinv_d03 = d03.ALT[0,:,:,:]
density_d03 = 1/densityinv_d03
airmass_d03 = np.multiply(density_d03 , dz_d03) * 3000*3000
print(airmass_d01.shape)
print(density_d01[:,0,0].data)

(29, 54, 61)
[ 1.1336236   1.12753999  1.11880648  1.10751879  1.09383094  1.07637084
  1.05039394  1.01596451  0.97282684  0.92774957  0.88192827  0.81890237
  0.74459511  0.67012399  0.60129738  0.53893423  0.48231423  0.43285158
  0.38820344  0.34808719  0.31212667  0.27907112  0.24857828  0.21735585
  0.18917671  0.1554485   0.13013634  0.10879468  0.09188558]


In [10]:
#extract all fractional water content
allwater_d01 = np.multiply(d01.QVAPOR[0,:,:,:]+d01.QCLOUD[0,:,:,:]+d01.QRAIN[0,:,:,:] \
                           +d01.QICE[0,:,:,:]+d01.QSNOW[0,:,:,:]+d01.QGRAUP[0,:,:,:],airmass_d01)
allwater_d02 = np.multiply(d02.QVAPOR[0,:,:,:]+d02.QCLOUD[0,:,:,:]+d02.QRAIN[0,:,:,:] \
                           +d02.QICE[0,:,:,:]+d02.QSNOW[0,:,:,:]+d02.QGRAUP[0,:,:,:],airmass_d02)
allwater_d03 = np.multiply(d03.QVAPOR[0,:,:,:]+d03.QCLOUD[0,:,:,:]+d03.QRAIN[0,:,:,:] \
                           +d03.QICE[0,:,:,:]+d03.QSNOW[0,:,:,:]+d03.QGRAUP[0,:,:,:],airmass_d03)

In [11]:
waterair_d01=sum(sum(sum(allwater_d01)))
waterrain_d01=sum(sum(ppt_d01[0]))*27000*27000
watertot_d01=(waterair_d01+waterrain_d01)
waterair_d02=sum(sum(sum(allwater_d02)))
waterrain_d02=sum(sum(ppt_d02[0]))*9000*9000
watertot_d02=(waterair_d02+waterrain_d02)
waterair_d03=sum(sum(sum(allwater_d03)))
waterrain_d03=sum(sum(ppt_d03[0]))*3000*3000
watertot_d03=(waterair_d03+waterrain_d03)
print('in air (QVAPOR, QCLOUD, QRAIN, QICE, QSNOW, QGRAUP) \n d01: {:.5e}kg \n d02: {:.5e}kg \n d03: {:.5e}kg' \
      .format(float(waterair_d01.data),float(waterair_d02.data),float(waterair_d03.data)))
print('as rain (RAINC, RAINNC) \n d01: {:.5e}kg \n d02: {:.5e}kg \n d03: {:.5e}kg' \
      .format(float(waterrain_d01),float(waterrain_d02),float(waterrain_d03)))
print('total \n d01: {:.5e}kg \n d02: {:.5e}kg \n d03: {:.5e}kg' \
      .format(float(watertot_d01.data),float(watertot_d02.data),float(watertot_d03.data)))

in air (QVAPOR, QCLOUD, QRAIN, QICE, QSNOW, QGRAUP) 
 d01: 1.39640e+14kg 
 d02: 1.40054e+14kg 
 d03: 1.39633e+14kg
as rain (RAINC, RAINNC) 
 d01: 5.48868e+12kg 
 d02: 5.68360e+12kg 
 d03: 3.43792e+12kg
total 
 d01: 1.45129e+14kg 
 d02: 1.45738e+14kg 
 d03: 1.43071e+14kg


in air (QCLOUD, QRAIN, QICE, QSNOW, QGRAUP) 
 d01: 6.32593e+11kg 
 d02: 7.29332e+11kg 
 d03: 6.72699e+11kg
as rain (RAINC, RAINNC) 
 d01: 3.27616e+12kg 
 d02: 2.05990e+12kg 
 d03: 1.46771e+12kg
total 
 d01: 3.90876e+12kg 
 d02: 2.78923e+12kg 
 d03: 2.14041e+12kg

in air (QVAPOR, QCLOUD, QRAIN, QICE, QSNOW, QGRAUP) 
 d01: 1.29266e+14kg 
 d02: 1.23309e+14kg 
 d03: 1.26153e+14kg
as rain (RAINC, RAINNC) 
 d01: 3.27616e+12kg 
 d02: 2.05990e+12kg 
 d03: 1.46771e+12kg
total 
 d01: 1.32542e+14kg 
 d02: 1.25369e+14kg 
 d03: 1.27621e+14kg

in air (QVAPOR) 
 d01: 1.28633e+14kg 
 d02: 1.22579e+14kg 
 d03: 1.25480e+14kg
as rain (RAINC, RAINNC) 
 d01: 3.27616e+12kg 
 d02: 2.05990e+12kg 
 d03: 1.46771e+12kg
total 
 d01: 1.31910e+14kg 
 d02: 1.24639e+14kg 
 d03: 1.26948e+14kg
 
 WITHOUT FDDA:
 in air (QVAPOR, QCLOUD, QRAIN, QICE, QSNOW, QGRAUP) 
 d01: 1.39640e+14kg 
 d02: 1.40054e+14kg 
 d03: 1.39633e+14kg
as rain (RAINC, RAINNC) 
 d01: 5.48868e+12kg 
 d02: 5.68360e+12kg 
 d03: 3.43792e+12kg
total 
 d01: 1.45129e+14kg 
 d02: 1.45738e+14kg 
 d03: 1.43071e+14kg
 strangely, the water problem kind of went away. Although there is still a drop in rainfall for the domain without parameterization.

In [77]:
print(type(waterrain_d02))

<class 'numpy.float64'>


From newest run
d01[ 4494.04999053  5847.97276306  6649.50907898  5637.32284164  5720.81284702
  7527.07045507  7986.60803032  8234.31759262]
d02[ 25430.89307904  37181.30161667  49642.04763031  42561.84369659
  40515.41278648  44375.89221096  47459.0758667   42719.67413425]
d03[ 163078.53554225  196817.87227154  282941.54413223  261391.75156975
  248790.2686823   289772.38904345  319332.56106257  319080.64634895]

RAINC+RAINNC: D01:39604.744612818584 D02:77326.6449586451 D03:132690.01867890358

In [ ]:
ax = plt.axes(projection=ccrs.Mercator())

plt.pcolormesh(d01.XLONG[0,0,:],d01.XLAT[0,:,0],d01.RAINC[0,:,:],transform=ccrs.Mercator())
ax.coastlines()
plt.show()


In [104]:
print(dz_d01[:,0,0].data)
print(dz_d02[:,0,0].data)
print(dz_d03[:,0,0].data)
print(zt_d01[:,0,0].data)
print(pblhr_d01[0,0,0].data)

[   0.            0.            0.            0.            0.            0.
  267.73614502  459.88183594  479.78686523  504.49243164  532.12988281
  993.11669922  994.27099609  994.21679688  994.98486328  996.45019531
  998.13574219  999.83105469  999.17480469  996.21386719  992.55957031
  991.17773438  963.296875    945.2578125   937.75292969  935.45117188
  941.36328125  964.88671875  997.24804688]
[    0.             0.             0.             0.             0.
   225.72747803   267.76776123   461.08642578   482.0279541    505.35522461
   532.65161133   993.95703125   992.72241211   991.99560547   994.48632812
   999.54931641  1003.74169922  1004.87890625  1000.65332031   997.734375
   996.00488281   996.55664062   961.203125     929.51269531   922.53320312
   942.2734375    949.33789062   950.49023438   981.2109375 ]
[    0.             0.             0.             0.             0.
   225.80682373   267.88110352   461.26635742   482.33691406   505.56311035
   532.89208984   9

(3, 3, 3) (3, 3, 3)
(3, 3, 3)


In [314]:
print(d01.P[0,:,0,0]+d01.PB[0,:,0,0])
print(d01.PB[0,0,0,0])
print(d01.P_TOP[0])
print('top pressure ',d01.P[0,-1,90,90].data+d01.PB[0,-1,90,90].data)

<xarray.DataArray (bottom_top: 29)>
array([ 100676.1171875 ,   99850.921875  ,   98734.6640625 ,
         97327.3515625 ,   95580.515625  ,   93397.7734375 ,
         90782.15625   ,   87060.5390625 ,   82419.5234375 ,
         77774.2421875 ,   73127.5       ,   66790.625     ,
         59145.17578125,   52225.72265625,   45976.41015625,
         40346.13671875,   35283.1640625 ,   30738.26757812,
         26666.203125  ,   23026.25195312,   19780.79882812,
         16895.1953125 ,   14377.82324219,   12230.140625  ,
         10402.63964844,    8847.61914062,    7524.42626953,
          6398.515625  ,    5440.44091797], dtype=float32)
Coordinates:
    XLAT         float32 6.66342
    XLONG        float32 95.6311
    XTIME        datetime64[ns] 2008-06-10
    Time         int64 0
    south_north  int64 0
    west_east    int64 0
  * bottom_top   (bottom_top) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 ...
<xarray.DataArray 'PB' ()>
array(99667.5, dtype=float32)
Coordinates:
    XLAT